In [1]:
import json
import pickle
from pathlib import Path
from typing import List

import cv2 as cv
import numpy as np
from sklearn.mixture import BayesianGaussianMixture

In [2]:
laser_samples_path = Path('spots')
pkl_model_path = Path('spot_model.pkl')
json_model_path = Path('spot_model.json')

In [3]:
laser_samples = laser_samples_path.rglob('*.jpg', case_sensitive=False)

In [4]:
imgs = [cv.imread(path) for path in laser_samples]

In [5]:
hsv_images: List[cv.Mat] = [cv.cvtColor(img, cv.COLOR_BGR2HSV) for img in imgs]

In [6]:
flattened_image = np.empty((0, 3), dtype=np.float32)
for img in hsv_images:
    flattened_image = np.vstack((flattened_image, img.reshape(-1, 3).astype(np.float32)))

In [7]:
gmm = BayesianGaussianMixture(
    n_components=10,
    max_iter=200,
    covariance_type='full'
)
gmm.fit(flattened_image)

BayesianGaussianMixture(max_iter=200, n_components=10)

In [8]:
with open(pkl_model_path, 'wb') as handle:
    pickle.dump(gmm, handle)

In [9]:
len(gmm.means_)

10

In [10]:
for cov in gmm.covariances_:
    print(cov)

[[ 312.99160757 -100.05145872 -119.27592868]
 [-100.05145872  175.96473637  171.57136102]
 [-119.27592868  171.57136102  265.9116234 ]]
[[  10.24395751   17.04676472   -3.19723172]
 [  17.04676472  944.63610108  518.8448217 ]
 [  -3.19723172  518.8448217  1022.76862752]]
[[1035.64778926 -359.85611685 -351.1549345 ]
 [-359.85611685  387.31549344  412.80745684]
 [-351.1549345   412.80745684  883.01442206]]
[[1035.64889374 -359.85622973 -351.15504605]
 [-359.85622973  387.31546635  412.80741189]
 [-351.15504605  412.80741189  883.01412239]]
[[1035.64999457 -359.85634223 -351.15515723]
 [-359.85634223  387.31543935  412.80736709]
 [-351.15515723  412.80736709  883.01382369]]
[[ 22.26347133 -22.85565438  14.74911114]
 [-22.85565438  94.98834953 -17.36211435]
 [ 14.74911114 -17.36211435 130.78730884]]
[[ 978.51507783 -261.61613246  -31.44182272]
 [-261.61613246  144.15889097   88.73683144]
 [ -31.44182272   88.73683144  590.80300134]]
[[ 805.684009   -140.7173778  -150.01598544]
 [-140.71737

In [11]:
with open(json_model_path, 'w', encoding='utf-8') as handle:
    json.dump({
        'weights': gmm.weights_.tolist(),
        'means': gmm.means_.tolist(),
        'covariances': gmm.covariances_.tolist()
    }, handle)